In [1]:
import datetime,os
nowtime = f'{datetime.datetime.now():%Y%m%d}'
one_week_ago = f'{datetime.datetime.now() - datetime.timedelta(days=7):%Y%m%d}'
print(nowtime)

20201124


In [2]:
os.makedirs('./wordcloud/'+ str(nowtime), exist_ok = True)

In [3]:
import tweepy
import json
    
#twitterAPIのためのconfigファイル
JSON_LOAD_FILE = r"./json/twitter.json"
#twitterからデータを抽出するときの検索ワード
SEARCH_WORD = "AYA世代"
#twitterからデータを抽出するときの取得件数
SEARCH_COUNT = 100

#tweeterAPIと繋げるAPIインスタンスを生成する
def twitter_api_connect():
    #twiteerのAPIを利用するためのキーを取得
    conf = json_load(JSON_LOAD_FILE)

    #twiteerのAPIを利用するための、認証データ作成
    auth = tweepy.OAuthHandler(conf['CONSUEMR_KEY'], conf['CONSUEMR_SECRET'])
    auth.set_access_token(conf['ACCESS_TOKEN_KEY'], conf['ACCESS_TOKEN_SECRET'])

    #twitterのAPIインスタンス生成
    twitter_api = tweepy.API(auth, wait_on_rate_limit = True)
    return twitter_api

#tweetのデータを取得し、リスト化し、返却する
def tweet_list_create(twitter_api, search_word, search_count):
    #カーソルを使用してデータ取得
    search_tweets = twitter_api.search(q = search_word, count = search_count,lang='ja')

    #tweetの内容を格納するためのリスト変数
    search_tweet_list = []

    #取得したtweetの内容をリストに格納
    for search_tweet in search_tweets:
        search_tweet_list.append(search_tweet.text) 
    return search_tweet_list

# JSONファイルの読み込み
def json_load(file_path):
    with open(file_path, 'r') as f:
        conf = json.load(f)
    return conf

#twitterのAPIインスタンス生成
twitter_api = twitter_api_connect()

#twitterからデータを取得
search_tweet_list = tweet_list_create(twitter_api, SEARCH_WORD, SEARCH_COUNT)

In [4]:
import MeCab

#形態素解析し、単語をリスト化するときの品詞の条件
GET_WORD_CLASS = ['名詞', '形容詞']

#mecabのインスタンス生成
mecab = MeCab.Tagger ('-Ochasen')
mecab.parse('')

def word_list_create(sentences, get_word_class):
    #複数文から、指定の品詞(GET_WORD_CLASS)を抽出した単語リスト
    sentences_word_list = []

    for sentence in sentences:
        #一文から、指定の品詞(GET_WORD_CLASS)を抽出した単語リスト
        one_sentence_word_list = []
        #形態素解析
        node = mecab.parseToNode(sentence)

        while node:

            #語幹
            word = node.feature.split(",")[-3]
            #品詞
            word_class = node.feature.split(",")[0]  
            #(指定の品詞(GET_WORD_CLASS)である) and (語幹が＊のもの(つまり未知語))場合は、単語リストに追加
            if word_class in get_word_class and word not in ['*','ん','こと','の',
                                                             "よう", "そう", "これ",
                                                             "それ",'さん']: 
                one_sentence_word_list.append(word) 

            node = node.next
        sentences_word_list.extend(one_sentence_word_list)
    return sentences_word_list

#twitterデータから、指定の品詞の単語リストを生成
word_list = word_list_create(search_tweet_list, GET_WORD_CLASS)

In [5]:
from collections import Counter
from matplotlib import pyplot as plt
from wordcloud import WordCloud
#wordcloud用のフォントパス
WORDCLOUD_FONT_PATH = '/home/lcj001/.fonts/NotoSansCJKjp-Regular.otf'
#wordcloud用の幅
WORDCLOUD_WIDTH = 800
#wordcloud用の高さ
WORDCLOUD_HEIGHT = 500
#wordcloud用の背景色
WORDCLOUD_BG_COLOR = 'white'


def word_cloud_png_creater(word_list, font_path, width, height, bg_color, search_word):
    word_counter = Counter(word_list)
    wc = WordCloud(font_path=font_path,
                   width=width, 
                   height=height, 
                   background_color=bg_color).generate_from_frequencies(word_counter)
    wc.to_file('./wordcloud/{0}/{1}_{0}.png'.format(nowtime,search_word))

#'./wordcloud/'フォルダに、検索文字列.pngでwordcloud作成
word_cloud_png_creater(word_list, WORDCLOUD_FONT_PATH, 
                       WORDCLOUD_WIDTH, WORDCLOUD_HEIGHT, WORDCLOUD_BG_COLOR, SEARCH_WORD)

In [6]:
search_tweet_list

['RT @3xLqH0zFfvo1VdY: GQJAPAN受賞式\n鈴木編集長のコメント文字起こし\n\nSixTONESに物凄く期待している\n今まで日本に無かったタイプのグループ\n新しい世代を代表してる。国際的な普遍性を持っていく。それを正面から見据えて音楽的にそれを目指してる。大変…',
 '今朝は念願叶いケアホーム西大井こうほうえんを視察して参りました！\n\n江東区では新砂保育園と高齢者施設を運営されている法人です。\n\n多世代型交流ができる施設としては勿論、廃校後の学校施設跡地活用としても大変参考になりました。\n\n西本… https://t.co/TZTbKVClCB',
 'RT @uedareiko: 【ケアホーム西大井こうほうえん視察】\n旧品川区立原小学校舎を活用してサービス付高齢者向け住宅、認可保育園、高齢者交流ひろば、体育館はスポーツと地域に開かれた多世代利用施設の視察を、WM議員の大先輩西本貴子品川区議のご案内、社会福祉士の三戸 安弥江東…',
 'RT @PinkRing5: Pink Ringより\n\n全国のAYA世代がん体験者の皆様へ\n（がん種・現在の年齢不問）マスクプレゼントのお知らせ🎀\n\n拡散大歓迎✨\n\n詳細は、HPをご覧下さい！\n\nhttps://t.co/Ea0fYr1QCX https://t.co/mFUu…',
 'RT @GenerationAya: 【12月5日(土) 9:00スタート】\n《AYA世代》をメインにしたプログラムです。\n\n＊参加無料 (応援チケットあり) ⬇︎\nhttps://t.co/ouxHHZMRN8\n\nAYAまつり専用プログラムサイト⬇︎\nhttps://t.co…',
 'Samplingmaster MEGAとかAYAっていうのはつまり「そういうこと」なんだと思います。この人たちFM音源現役世代ですし。恐ろしい！',
 'RT @Sponichi_Tfoto: 試合前に同世代の ＃山瀬慎之助 さんと談笑する ＃井上広大 さんです。 https://t.co/YTFHBlnXd5',
 '@aya__ss_ ホントなんだろ涙が気づいたら出てる\u200dｗｗｗ\nなんで、私ら同世代じゃないんだろ😅',
 'RT @PinkRing5: Pink Ring

In [7]:
word_list

['ReTweet',
 '1V',
 '受賞',
 '式',
 '鈴木',
 '編集',
 '長',
 'コメント',
 '文字起こし',
 'SixTONES',
 '物凄い',
 '期待',
 '今',
 '日本',
 '無い',
 'タイプ',
 'グループ',
 '新しい',
 '世代',
 '代表',
 '国際',
 '的',
 '普遍',
 '性',
 '正面',
 '音楽的',
 '大変',
 '今朝',
 '念願',
 'ケア',
 'ホーム',
 '西大井',
 '豊艷',
 '視察',
 '江東区',
 '新砂',
 '保育園',
 '高齢者施設',
 '運営',
 '法人',
 '世代',
 '型',
 '交流',
 '施設',
 '廃校',
 '後',
 '学校施設',
 '跡地',
 '活用',
 '大変',
 '参考',
 '西本',
 'HTTPS',
 'ReTweet',
 'ケア',
 'ホーム',
 '西大井',
 '豊艷',
 '視察',
 '品川区',
 '立原',
 '小学校',
 '舎',
 '活用',
 'サービス',
 '付',
 '高齢者',
 '向け',
 '住宅',
 '認可',
 '保育園',
 '高齢者',
 '交流',
 'ひろば',
 '体育館',
 'スポーツ',
 '地域',
 '世代',
 '利用',
 '施設',
 '視察',
 '議員',
 '先輩',
 '西本',
 '貴子',
 '品川区',
 '議',
 '案内',
 '社会福祉',
 '士',
 '三戸',
 '安弥',
 '江東',
 'ReTweet',
 'PINK',
 'Rin',
 'G5',
 'PINK',
 'Ring',
 '全国',
 'AYA世代',
 'がん',
 '体験',
 '者',
 '皆様',
 'がん',
 '種',
 '現在',
 '年齢',
 '不問',
 'マスク',
 'プレゼント',
 'お知らせ',
 '拡散',
 '歓迎',
 '詳細',
 'HP',
 'ご覧',
 'HTTPS',
 '第1四半期',
 'CX',
 'HTTPS',
 'ReTweet',
 '12月5日',
 '土',
 'スタート',
 'AYA世代',
 'メイン',
 'プログラム',
 '参加無料',


In [8]:
#必要なライブラリをimport
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import joblib
import numpy as np

In [9]:
lda = LatentDirichletAllocation(n_components=3, n_jobs=-1, verbose=1)
word_list_2d = np.array([word_list])
word_list_2d = word_list_2d.reshape(1.-1)
X_transformed = lda.fit_transform(word_list)

TypeError: 'float' object cannot be interpreted as an integer